In [1]:
from transformers import BertForSequenceClassification, BertTokenizerFast, pipeline
from utils.data_utils import read_data
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# train_dataset, test_dataset, dev_dataset = read_data("../data/all/combined_lfud_huggingface_nonfallacies.csv")
train_dataset, test_dataset, dev_dataset = read_data("../data/all/combined_lfud_huggingface_binary.csv")

In [ ]:
# model_path = "../model/outputs/21-02-2025_14-45-55_bert-2-classes-model.pickle" 
# model_path = "../model/outputs/03-03-2025_16-23-08_bert-3-classes-model.pickle"
# model_path = "../model/outputs/03-03-2025_16-46-39_bert-5-classes-model.pickle" 
# model_path = "../model/outputs/20-02-2025_10-26-38_bert-all-classes-model.pickle" 
model_path = "../model/outputs/29-03-2025_14-38-54_bert-3-classes-model.pickle" 


model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
# logical_fallacies = ['nonfallacy', 'fallacy']
logical_fallacies = ['nonfallacy', 'faulty generalization', 'intentional']
# logical_fallacies = ['nonfallacy', 'faulty generalization', 'intentional', 'ad hominem', 'false causality']
# logical_fallacies = list(set(list(test_dataset['logical_fallacies'])))
filtered_test_data = test_dataset[test_dataset.logical_fallacies.isin(logical_fallacies)]
filtered_test_data

In [ ]:
filtered_test_data["logical_fallacies"].value_counts()

In [ ]:
predictions = []
for data in filtered_test_data["source_article_ro"]:
    predictions.append(nlp(data)[0]["label"])

In [2]:
report = classification_report(filtered_test_data["logical_fallacies"], predictions)
accuracy = accuracy_score(filtered_test_data["logical_fallacies"], predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

NameError: name 'filtered_test_data' is not defined

In [ ]:
import numpy as np

y_true = np.array(filtered_test_data["logical_fallacies"])
y_pred = np.array(predictions)

classes = np.unique(y_true) 

for cls in classes:
    cls_mask = (y_true == cls)
    cls_correct = (y_true[cls_mask] == y_pred[cls_mask])
    cls_accuracy = cls_correct.sum() / cls_mask.sum()
    print(f"Class {cls} Accuracy: {cls_accuracy:.2f}")


Confusion Matrix

In [ ]:
class_labels = sorted(logical_fallacies)  # Ensure consistent ordering

cm = confusion_matrix(filtered_test_data["logical_fallacies"], predictions, labels=class_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


### Validation - sentiment

In [12]:
from utils.data_utils import read_data

train_dataset, test_dataset, dev_dataset = read_data("../data/all/combined_lfud_huggingface_nonfallacies_sent.csv", sentiment = True)

# logical_fallacies = ['nonfallacy', 'fallacy']
# logical_fallacies = ['nonfallacy', 'faulty generalization', 'intentional']
# logical_fallacies = ['nonfallacy', 'faulty generalization', 'intentional', 'ad hominem', 'false causality']
logical_fallacies = list(set(list(train_dataset['logical_fallacies'])))

filtered_test_data = test_dataset[test_dataset.logical_fallacies.isin(logical_fallacies)]

In [13]:
filtered_test_data

,logical_fallacies,source_article_ro,sentiment
4279,false dilemma,"Dacă este știință, nu este un consens.",neutral
4280,faulty generalization,"„Dacă nu obții un A la clasa domnului K, vei p...",negative
4281,false causality,"De fiecare dată când port acest colier, îmi pr...",neutral
4282,fallacy of logic,Forarea carotelor de gheață arată că la 800 de...,positive
4283,fallacy of credibility,Nu este timpul să începem să ignorăm afirmații...,negative
...,...,...,...
5129,fallacy of relevance,O eroare logică care compară abaterile minore ...,negative
5130,appeal to emotion,Este deja aici. Și se va înrăutăți.,negative
5131,intentional,Poate cercetarea climatică să revină pe drumul...,negative
5132,faulty generalization,Toate cursurile universitare necesare sunt pli...,negative


In [14]:
from transformers import BertModel
import torch.nn as nn

sentiment_mapping = {"negative": 0, "neutral": 1, "positive": 2}

class BertWithSentiment(nn.Module):
    def __init__(self, model_name, num_labels, num_sentiment_classes=3):
        super(BertWithSentiment, self).__init__()
        self.bert = BertModel.from_pretrained(model_name, return_dict=True)  # Ensure return_dict=True
        self.sentiment_embedding = nn.Embedding(num_sentiment_classes, 768)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768 * 2, num_labels)

    def forward(self, input_ids, attention_mask, sentiment, labels=None):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token_embedding = bert_output.last_hidden_state[:, 0, :]
        sentiment_embed = self.sentiment_embedding(sentiment)
        combined = torch.cat((cls_token_embedding, sentiment_embed), dim=1)

        logits = self.classifier(self.dropout(combined))

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)  # Ensure labels are (batch_size,) with class indices

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [ ]:
import torch
from transformers import BertTokenizer

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "../model/outputs/outputs/model.pt"
tokenizer_path = "../model/outputs/outputs/tokenizer"

model_name = "dumitrescustefan/bert-base-romanian-uncased-v1"

# Get number of labels (you must know your training labels)
# logical_fallacies = list(set(list(filtered_test_data['logical_fallacies'])))
label2id = {label: id for id, label in enumerate(logical_fallacies)}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

# Recreate model and load weights
model = BertWithSentiment(model_name=model_name, num_labels=num_labels)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

# Define sentiment mapping
sentiment_mapping = {"negative": 0, "neutral": 1, "positive": 2}

# Tokenize test data
def tokenize_function(texts, sentiments, tokenizer):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    sentiment_ids = torch.tensor([sentiment_mapping[s] for s in sentiments])
    return inputs, sentiment_ids

# Prepare dataset
texts = filtered_test_data["source_article_ro"].tolist()
sentiments = filtered_test_data["sentiment"].tolist()
labels = filtered_test_data["logical_fallacies"].tolist()  # Optional

# Tokenize
inputs, sentiment_ids = tokenize_function(texts, sentiments, tokenizer)
inputs = {key: val.to(device) for key, val in inputs.items()}
sentiment_ids = sentiment_ids.to(device)

# Remove token_type_ids
if "token_type_ids" in inputs:
    inputs.pop("token_type_ids")

# Run inference
with torch.no_grad():
    outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], sentiment=sentiment_ids)
    predictions = torch.argmax(outputs["logits"], dim=1)

# Convert predictions to labels
predicted_labels = [id2label[pred.item()] for pred in predictions]

predicted_labels

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

report = classification_report(filtered_test_data["logical_fallacies"], predicted_labels)
accuracy = accuracy_score(filtered_test_data["logical_fallacies"], predicted_labels)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
import numpy as np

y_true = np.array(filtered_test_data["logical_fallacies"])
y_pred = np.array(predicted_labels)

classes = np.unique(y_true)

for cls in classes:
    cls_mask = (y_true == cls)
    cls_correct = (y_true[cls_mask] == y_pred[cls_mask])
    cls_accuracy = cls_correct.sum() / cls_mask.sum()
    print(f"Class {cls} Accuracy: {cls_accuracy:.2f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_labels = sorted(logical_fallacies)  # Ensure consistent ordering

cm = confusion_matrix(filtered_test_data["logical_fallacies"], predicted_labels, labels=class_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()